# 🚀 RunPod Pod Management Test Notebook

Bu notebook, RunPod API kullanarak pod oluşturma, yönetme ve SSH erişimi sağlama süreçlerini test eder.

## 📋 Test Kapsamı:
- ✅ **API Bağlantısı**: GraphQL API ile iletişim
- ✅ **Pod Oluşturma**: GPU ile pod deployment
- ✅ **Durum İzleme**: Sabırlı bekleyiş sistemi 
- ✅ **Bağlantı Bilgileri**: Jupyter ve SSH URL'leri
- ✅ **Erişim Testi**: Notebook ve terminal bağlantıları

## 🔧 Gereksinimler:
- RunPod API Key (`.env` dosyasında)
- Python 3.10+ 
- İnternet bağlantısı

In [ ]:
# 📦 1. Gerekli Kütüphaneleri İçe Aktar
import os
import time
import requests
from typing import Dict, Any
from dotenv import load_dotenv
import json

print("✅ Tüm kütüphaneler başarıyla yüklendi!")
print("🔍 Python versiyonu:", __import__('sys').version)
print("🌐 Requests versiyonu:", requests.__version__)

In [ ]:
# 🔐 2. Ortam Değişkenleri ve API Yapılandırması
load_dotenv()

API_KEY = os.getenv("RUNPOD_API_KEY")
GRAPHQL_URL = "https://api.runpod.io/graphql"

if not API_KEY:
    print("❌ HATA: RUNPOD_API_KEY bulunamadı!")
    print("🔧 Çözüm: .env dosyanızda RUNPOD_API_KEY=your_key_here ekleyin")
else:
    print("✅ API Key başarıyla yüklendi!")
    print(f"🔗 GraphQL Endpoint: {GRAPHQL_URL}")
    print(f"🔑 API Key (ilk 10 karakter): {API_KEY[:10]}...")

HEADERS = {"Authorization": f"Bearer {API_KEY}"}

In [ ]:
# 🔧 3. GraphQL Yardımcı Fonksiyonları
def run_graphql_query(query: str, variables: Dict = None) -> Dict:
    """GraphQL sorgusu çalıştırmak için yardımcı fonksiyon."""
    try:
        response = requests.post(
            GRAPHQL_URL, 
            json={"query": query, "variables": variables or {}}, 
            headers=HEADERS
        )
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ GraphQL isteği sırasında hata: {e}")
        if hasattr(e, 'response') and e.response:
            print(f"📄 Hata Detayı: {e.response.text}")
        return {"errors": [{"message": str(e)}]}

# Test API bağlantısı
print("🧪 API bağlantısı test ediliyor...")
test_query = """
query {
    myself {
        id
        email
    }
}
"""

test_result = run_graphql_query(test_query)
if "errors" in test_result:
    print(f"❌ API bağlantı hatası: {test_result['errors'][0]['message']}")
else:
    user_data = test_result.get("data", {}).get("myself", {})
    print(f"✅ API bağlantısı başarılı!")
    print(f"👤 Kullanıcı ID: {user_data.get('id', 'N/A')}")
    print(f"📧 Email: {user_data.get('email', 'N/A')}")

In [ ]:
# 🚀 4. SSH ve Jupyter Yapılandırması ile Pod Oluşturma
def create_pod_with_jupyter(gpu_type_id: str) -> Dict:
    """SSH ve Jupyter ile pod oluşturur."""
    print(f"🔥 Pod oluşturuluyor...")
    print(f"🎮 GPU Type: {gpu_type_id}")
    
    mutation = """
    mutation PodFindAndDeployOnDemand($input: PodFindAndDeployOnDemandInput!) {
      podFindAndDeployOnDemand(input: $input) {
        id
        imageName
        machineId
      }
    }
    """
    
    variables = {
        "input": {
            "cloudType": "COMMUNITY",
            "gpuTypeId": gpu_type_id,
            "name": f"notebook-test-{int(time.time())}",
            "imageName": "runpod/pytorch:2.1.0-py3.10-cuda11.8.0-devel-ubuntu22.04",
            "gpuCount": 1,
            "volumeInGb": 40,
            "volumeMountPath": "/workspace",
            "containerDiskInGb": 10,
            "startSsh": True,
            "startJupyter": True,
            "ports": "8888/http",
            "env": [{"key": "JUPYTER_PASSWORD", "value": "atolye123"}]
        }
    }
    
    result = run_graphql_query(mutation, variables)
    
    if "errors" in result:
        print(f"❌ Pod oluşturma hatası: {result['errors'][0].get('message')}")
        return {"status": "error", "message": result['errors'][0].get('message')}
    
    pod_data = result.get("data", {}).get("podFindAndDeployOnDemand")
    if not pod_data:
        print("❌ Pod oluşturulamadı - uygun GPU bulunamamış olabilir.")
        return {"status": "error", "message": "Pod oluşturulamadı"}
    
    pod_id = pod_data.get("id")
    image_name = pod_data.get("imageName")
    print(f"✅ Pod başarıyla oluşturuldu!")
    print(f"🆔 Pod ID: {pod_id}")
    print(f"🖼️ Image: {image_name}")
    
    return {
        "status": "success",
        "pod_id": pod_id,
        "image_name": image_name,
        "machine_id": pod_data.get("machineId")
    }

# Test için GPU türü
GPU_TYPE = "NVIDIA GeForce RTX 3070"
print(f"🎯 Test için seçilen GPU: {GPU_TYPE}")
print("⚠️ Not: Bu hücreyi çalıştırmak yeni bir pod oluşturacak!")

In [ ]:
# ⏳ 5. Pod Durumu İzleme ve Hazır Olma Bekleyişi
def wait_for_pod_ready(pod_id: str, max_attempts: int = 15) -> Dict:
    """Pod'un Jupyter port 8888'inin hazır olmasını bekler."""
    print(f"📡 Pod '{pod_id}' hazırlık durumu izleniyor...")
    print(f"⏰ Maksimum {max_attempts} deneme, 15 saniye aralıklarla")
    
    status_query = """
    query pods {
        myself {
            pods {
                id
                runtime {
                    uptimeInSeconds
                    ports {
                        ip
                        isIpPublic
                        privatePort
                        publicPort
                        type
                    }
                }
            }
        }
    }
    """
    
    for attempt in range(max_attempts):
        print(f"🔍 Deneme [{attempt + 1}/{max_attempts}] - Pod durumu kontrol ediliyor...")
        
        status_result = run_graphql_query(status_query, {})
        
        if "errors" in status_result:
            print(f"❌ Durum sorgusu hatası: {status_result['errors'][0]['message']}")
            continue
            
        # Pod'umuzu bulalım
        pods = status_result.get("data", {}).get("myself", {}).get("pods", [])
        current_pod = None
        for pod in pods:
            if pod.get("id") == pod_id:
                current_pod = pod
                break
        
        if not current_pod:
            print("⚠️ Pod bulunamadı, devam ediliyor...")
            time.sleep(15)
            continue
            
        runtime = current_pod.get("runtime")
        if not runtime:
            print("⏳ Pod henüz runtime bilgisine sahip değil...")
            time.sleep(15)
            continue
            
        uptime = runtime.get("uptimeInSeconds", 0)
        ports = runtime.get("ports", [])
        
        print(f"📊 Pod durumu: {len(ports)} port, çalışma süresi: {uptime}s")
        
        # Port 8888'i arıyoruz
        jupyter_ready = False
        for port in ports:
            if port.get("privatePort") == 8888:
                jupyter_url = f"https://{pod_id}-8888.proxy.runpod.net/lab/"
                print(f"✅ Jupyter Notebook hazır!")
                print(f"🔗 URL: {jupyter_url}")
                print(f"🔐 Şifre: atolye123")
                return {
                    "status": "ready",
                    "jupyter_url": jupyter_url,
                    "uptime": uptime,
                    "total_ports": len(ports)
                }
        
        print(f"⏳ Jupyter portu (8888) henüz hazır değil...")
        if attempt < max_attempts - 1:
            print(f"⏰ 15 saniye bekleniyor...")
            time.sleep(15)
    
    return {
        "status": "timeout",
        "message": f"Pod {max_attempts * 15} saniye içinde hazır hale gelmedi"
    }

print("✅ Pod izleme fonksiyonu tanımlandı!")
print("📋 Kullanım: wait_for_pod_ready(pod_id)")

In [ ]:
# 📋 6. Mevcut Pod'ları Listele ve Hızlı Erişim
def list_all_pods():
    """Tüm pod'ları listeler."""
    query = """
    query {
        myself {
            pods {
                id
                name
                desiredStatus
                runtime {
                    uptimeInSeconds
                }
            }
        }
    }
    """
    
    result = run_graphql_query(query)
    
    if "errors" in result:
        print(f"❌ Pod listesi alınamadı: {result['errors'][0]['message']}")
        return
    
    pods = result.get("data", {}).get("myself", {}).get("pods", [])
    
    if not pods:
        print("📝 Hiç pod bulunamadı.")
        return
    
    print("📋 MEVCUT POD'LAR:")
    print("=" * 60)
    
    for i, pod in enumerate(pods, 1):
        pod_id = pod.get("id", "")
        pod_name = pod.get("name", "")
        status = pod.get("desiredStatus", "")
        runtime = pod.get("runtime", {})
        uptime = runtime.get("uptimeInSeconds", 0) if runtime else 0
        
        print(f"{i}. 🆔 {pod_id}")
        print(f"   📛 Name: {pod_name}")
        print(f"   🟢 Status: {status}")
        print(f"   ⏱️ Uptime: {uptime}s")
        
        # Jupyter URL'i oluştur
        if uptime > 0:  # Pod çalışıyorsa
            jupyter_url = f"https://{pod_id}-8888.proxy.runpod.net/lab/"
            print(f"   🔗 Jupyter: {jupyter_url}")
        
        print()
    
    print("=" * 60)

# Pod'ları listele
print("🔍 Mevcut pod'ları listeleniyor...")
list_all_pods()

In [ ]:
# 🧪 7. Tam Pod Oluşturma ve Erişim Testi
def test_complete_workflow():
    """Tam iş akışını test eder: Pod oluştur -> Bekle -> Bağlan."""
    print("🚀 TAM İŞ AKIŞI TESTİ BAŞLIYOR")
    print("=" * 50)
    
    # 1. Pod Oluştur
    print("\n1️⃣ Pod oluşturuluyor...")
    create_result = create_pod_with_jupyter(GPU_TYPE)
    
    if create_result["status"] != "success":
        print(f"❌ Pod oluşturma başarısız: {create_result['message']}")
        return create_result
    
    pod_id = create_result["pod_id"]
    print(f"✅ Pod oluşturuldu: {pod_id}")
    
    # 2. Hazır olmasını bekle
    print("\n2️⃣ Pod'un hazır olması bekleniyor...")
    wait_result = wait_for_pod_ready(pod_id)
    
    if wait_result["status"] != "ready":
        print(f"❌ Pod hazır hale gelmedi: {wait_result['message']}")
        return wait_result
    
    print("\n✅ TAM İŞ AKIŞI TESTİ BAŞARILI!")
    print("🎯 Jupyter Notebook'a erişebilirsiniz!")
    print(f"🔗 URL: {wait_result['jupyter_url']}")
    print(f"🔐 Şifre: atolye123")
    
    return {
        "status": "success",
        "pod_id": pod_id,
        "jupyter_url": wait_result["jupyter_url"]
    }

# Manuel test çalıştırma
print("🎮 Manuel Test Hazır!")
print("⚠️ Dikkat: test_complete_workflow() çalıştırmak yeni bir pod oluşturacak!")
print("💰 Bu işlem ücretlendirilir!")
print()
print("📋 Test çalıştırmak için:")
print("   test_result = test_complete_workflow()")
print()
print("🔍 Mevcut pod'ları kontrol etmek için:")
print("   list_all_pods()")